In [1]:
!git clone https://github.com/davide-coccomini/Combining-EfficientNet-and-Vision-Transformers-for-Video-Deepfake-Detection.git
%cd Combining-EfficientNet-and-Vision-Transformers-for-Video-Deepfake-Detection


Cloning into 'Combining-EfficientNet-and-Vision-Transformers-for-Video-Deepfake-Detection'...
remote: Enumerating objects: 162, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 162 (delta 28), reused 14 (delta 14), pack-reused 123 (from 1)
Receiving objects: 100% (162/162), 1.10 MiB | 8.27 MiB/s, done.
Resolving deltas: 100% (53/53), done.
/kaggle/working/Combining-EfficientNet-and-Vision-Transformers-for-Video-Deepfake-Detection


In [8]:
# Step 1: Install necessary libraries
!pip install efficientnet_pytorch einops timm
!pip install facenet_pytorch
# !pip install insightface onnxruntime
!pip install pillow==9.5.0
!pip install insightface==0.7.3 onnxruntime






  Using cached pillow-10.2.0-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (9.7 kB)
Using cached pillow-10.2.0-cp311-cp311-manylinux_2_28_x86_64.whl (4.5 MB)
  Attempting uninstall: Pillow
    Found existing installation: Pillow 9.5.0
    Uninstalling Pillow-9.5.0:
      Successfully uninstalled Pillow-9.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dopamine-rl 4.1.2 requires gymnasium>=1.0.0, but you have gymnasium 0.29.0 which is incompatible.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
plotnine 0.14.5 requires matplotlib>=3.8.0, but you have matplotlib 3.7.2 which is incompatible.
mlxtend 0.23.4 requires scikit-learn>=1.3.1, but you have scikit-learn 1.2.2 which is incompatible.
  Using cached Pillow-9.5.0-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (9.5 kB)
Using cached Pillow-9.5.0-cp311-cp311-manyl

In [5]:
!pip install "pillow<10"


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 39.2 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: pillow
    Found existing installation: pillow 10.2.0
    Uninstalling pillow-10.2.0:
      Successfully uninstalled pillow-10.2.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
facenet-pytorch 2.6.0 requires Pillow<10.3.0,>=10.2.0, but you have pillow 9.5.0 which is incompatible.
dopamine-rl 4.1.2 requires gymnasium>=1.0.0, but you have gymnasium 0.29.0 which is incompatible.
scikit-image 0.25.2 requires pillow>=10.1, but you have pillow 9.5.0 which is incompatible.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
plotnine 0.14.5 requires matplotlib>=3.8.0, but you have matplotlib 3.7.2 which is incompatible.
mlxtend 0.23.4 requires scikit-learn>=1.3.1, but you have scikit-learn 1.2.2 which is incompatible.


In [9]:

# Step 2: Imports
import os
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import numpy as np
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import matplotlib.pyplot as plt
import random
import timm
from efficientnet_pytorch import EfficientNet
from einops import rearrange
from tqdm import tqdm
from torchvision.transforms.functional import to_pil_image

In [10]:
# Step 3: Config 
class DotDict(dict):
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

config = DotDict({
    'model': DotDict({
        'image_size': 224,
        'patch_size': 16,
        'in_channels': 3,
        'emb_dim': 512,
        'num_classes': 2,
        'dropout': 0.1
    }),
    'training': DotDict({
        'batch_size': 32,
        'lr': 0.001,
        'weight_decay': 0.0001,
        'step_size': 5,
        'gamma': 0.5
    })
})

In [13]:
import os, shutil
from sklearn.model_selection import train_test_split

SOURCE_DIR = "/kaggle/input/archive-try/uadfv"
DEST_DIR = "/kaggle/working/split_dataset"
CLASSES = ["real", "fake"]
SPLITS = ["train", "val", "test"]

# Create split directories
for split in SPLITS:
    for cls in CLASSES:
        os.makedirs(os.path.join(DEST_DIR, split, cls), exist_ok=True)

def split_and_copy(cls):
    files = sorted(os.listdir(os.path.join(SOURCE_DIR, cls)))
    train_val, test = train_test_split(files, test_size=0.15, random_state=42)
    train, val = train_test_split(train_val, test_size=0.176, random_state=42)  # ~15% val

    for split, split_files in zip(SPLITS, [train, val, test]):
        for f in split_files:
            shutil.copy(os.path.join(SOURCE_DIR, cls, f), os.path.join(DEST_DIR, split, cls, f))

for cls in CLASSES:
    split_and_copy(cls)


In [15]:
import cv2
import os
import numpy as np
import torch
from tqdm import tqdm
from PIL import Image
import insightface

# Set device
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Initialize InsightFace model
model = insightface.app.FaceAnalysis(name='buffalo_l')
model.prepare(ctx_id=0 )  # Use 0 for GPU, -1 for CPU

# Define base paths
SPLITS = ['train', 'val', 'test']
INPUT_BASE = '/kaggle/working/split_dataset'
OUTPUT_BASE = '/kaggle/working/cropped_faces'

# Loop through splits and classes
for split in SPLITS:
    for label in ['real', 'fake']:
        input_folder = os.path.join(INPUT_BASE, split, label)
        output_folder = os.path.join(OUTPUT_BASE, split, label)
        os.makedirs(output_folder, exist_ok=True)

        video_files = [f for f in os.listdir(input_folder) if f.endswith(('.mp4', '.avi', '.mov'))]

        for video_name in tqdm(video_files, desc=f"{split}/{label}"):
            video_path = os.path.join(input_folder, video_name)
            cap = cv2.VideoCapture(video_path)
            
            frame_idx = 0
            saved_faces = 0

            while True:
                ret, frame = cap.read()
                if not ret:
                    break

                # Sample every 10th frame
                if frame_idx % 10 == 0:
                    # InsightFace expects BGR directly
                    faces = model.get(frame)
                    
                    for i, face in enumerate(faces):
                        x1, y1, x2, y2 = face.bbox.astype(int)

                        # Optional: add margin
                        margin_ratio = 0.2
                        w, h = x2 - x1, y2 - y1
                        x1 = max(0, int(x1 - w * margin_ratio))
                        y1 = max(0, int(y1 - h * margin_ratio))
                        x2 = min(frame.shape[1], int(x2 + w * margin_ratio))
                        y2 = min(frame.shape[0], int(y2 + h * margin_ratio))

                        face_crop = frame[y1:y2, x1:x2]

                        # Save face crop
                        face_path = os.path.join(output_folder, f"{video_name}_{frame_idx}_{i}.jpg")
                        cv2.imwrite(face_path, face_crop)
                        saved_faces += 1

                frame_idx += 1

            cap.release()


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)


test/fake: 100%|██████████| 8/8 [03:22<00:00, 25.25s/it]


In [16]:
# Step 4: Dataset Class
class CustomDataset(Dataset):
    def __init__(self, image_folder, transform=None):
        self.image_folder = image_folder
        self.transform = transform
        self.images = []
        self.labels = []
        
        for label_folder in os.listdir(image_folder):
            label_path = os.path.join(image_folder, label_folder)
            label = 0 if label_folder == 'real' else 1
            for img_file in os.listdir(label_path):
                self.images.append(os.path.join(label_path, img_file))
                self.labels.append(label)
                
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        image = Image.open(self.images[idx]).convert('RGB')
        if self.transform:
            image = self.transform(image)
        label = self.labels[idx]
        return image, label

In [17]:
# Step 5: Model
class PatchEmbedding(nn.Module):
    def __init__(self, image_size, patch_size, in_channels, emb_dim):
        super().__init__()
        assert image_size % patch_size == 0, "Image size must be divisible by patch size"
        self.num_patches = (image_size // patch_size) ** 2
        self.proj = nn.Conv2d(in_channels, emb_dim, kernel_size=patch_size, stride=patch_size)
        self.pos_embed = nn.Parameter(torch.zeros(1, self.num_patches, emb_dim))

    def forward(self, x):
        x = self.proj(x)  # [B, emb_dim, H/patch, W/patch]
        x = x.flatten(2).transpose(1, 2)  # [B, num_patches, emb_dim]
        x = x + self.pos_embed  # Add positional encoding
        return x
    
class TransformerEncoder(nn.Module):
    def __init__(self, emb_dim, num_heads=8, ff_hidden_mult=4, dropout=0.1):
        super().__init__()
        self.ln1 = nn.LayerNorm(emb_dim)
        self.msa = nn.MultiheadAttention(emb_dim, num_heads, batch_first=True)
        self.ln2 = nn.LayerNorm(emb_dim)
        self.mlp = nn.Sequential(
            nn.Linear(emb_dim, ff_hidden_mult * emb_dim),
            nn.GELU(),
            nn.Linear(ff_hidden_mult * emb_dim, emb_dim),
            nn.Dropout(dropout)
        )
    
    def forward(self, x):
        x_res = x + self.msa(self.ln1(x), self.ln1(x), self.ln1(x))[0]
        x_res = x_res + self.mlp(self.ln2(x_res))
        return x_res

class EfficientViT(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.efficientnet = EfficientNet.from_pretrained('efficientnet-b0')
        self.efficientnet._fc = nn.Identity()
        self.patch_embed = PatchEmbedding(
            image_size=config.image_size,
            patch_size=config.patch_size,
            in_channels=config.in_channels,
            emb_dim=config.emb_dim
        )
        self.transformer = TransformerEncoder(emb_dim=config.emb_dim, num_heads=8, dropout=config.dropout)
        self.fc = nn.Linear(config.emb_dim + 1280, config.num_classes)

    def forward(self, x):
        eff_out = self.efficientnet.extract_features(x)
        eff_out = nn.functional.adaptive_avg_pool2d(eff_out, 1).squeeze(-1).squeeze(-1)
        vit_input = self.patch_embed(x)
        vit_out = self.transformer(vit_input).mean(dim=1)
        combined = torch.cat((eff_out, vit_out), dim=1)
        return self.fc(combined)


In [18]:
# Step 6: Load Data
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Replace with your actual dataset path on Kaggle
train_dataset = CustomDataset("/kaggle/working/cropped_faces/train", transform=transform)
test_dataset = CustomDataset("/kaggle/working/cropped_faces/test", transform=transform)

train_loader = DataLoader(train_dataset, batch_size=config.training.batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=config.training.batch_size, shuffle=False)


In [19]:
# Create checkpoint folder
checkpoint_dir = "/kaggle/working/checkpoints"
os.makedirs(checkpoint_dir, exist_ok=True)


In [26]:
%%writefile /kaggle/working/Combining-EfficientNet-and-Vision-Transformers-for-Video-Deepfake-Detection/efficient-vit/efficient_vit.py


import torch
from torch import nn, einsum
from einops import rearrange
from efficientnet_pytorch import EfficientNet

class Residual(nn.Module):
    def __init__(self, fn):
        super().__init__()
        self.fn = fn

    def forward(self, x, **kwargs):
        return self.fn(x, **kwargs) + x

class PreNorm(nn.Module):
    def __init__(self, dim, fn):
        super().__init__()
        self.norm = nn.LayerNorm(dim)
        self.fn = fn

    def forward(self, x, **kwargs):
        return self.fn(self.norm(x), **kwargs)

class FeedForward(nn.Module):
    def __init__(self, dim, hidden_dim, dropout=0.):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, dim),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        return self.net(x)

class Attention(nn.Module):
    def __init__(self, dim, heads=8, dim_head=64, dropout=0.):
        super().__init__()
        inner_dim = dim_head * heads
        self.heads = heads
        self.scale = dim_head ** -0.5

        self.attend = nn.Softmax(dim=-1)
        self.to_qkv = nn.Linear(dim, inner_dim * 3, bias=False)
        self.to_out = nn.Sequential(
            nn.Linear(inner_dim, dim),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        b, n, _, h = *x.shape, self.heads
        qkv = self.to_qkv(x).chunk(3, dim=-1)
        q, k, v = map(lambda t: rearrange(t, 'b n (h d) -> b h n d', h=h), qkv)

        dots = einsum('b h i d, b h j d -> b h i j', q, k) * self.scale
        attn = self.attend(dots)

        out = einsum('b h i j, b h j d -> b h i d', attn, v)
        out = rearrange(out, 'b h n d -> b n (h d)')
        return self.to_out(out)

class Transformer(nn.Module):
    def __init__(self, dim, depth, heads, dim_head, mlp_dim, dropout=0.):
        super().__init__()
        self.layers = nn.ModuleList([
            nn.ModuleList([
                PreNorm(dim, Attention(dim, heads=heads, dim_head=dim_head, dropout=dropout)),
                PreNorm(dim, FeedForward(dim, mlp_dim, dropout=dropout))
            ]) for _ in range(depth)
        ])

    def forward(self, x):
        for attn, ff in self.layers:
            x = attn(x) + x
            x = ff(x) + x
        return x

class EfficientViT(nn.Module):
    def __init__(self, config):
        super().__init__()

        image_size = config['model']['image-size']
        patch_size = config['model']['patch-size']
        num_classes = config['model']['num-classes']
        dim = config['model']['dim']
        depth = config['model']['depth']
        heads = config['model']['heads']
        mlp_dim = config['model']['mlp-dim']
        emb_dim = config['model']['emb-dim']
        dim_head = config['model']['dim-head']
        dropout = config['model']['dropout']
        emb_dropout = config['model']['emb-dropout']

        assert image_size % patch_size == 0, 'image dimensions must be divisible by the patch size'

        self.efficient_net = EfficientNet.from_pretrained('efficientnet-b0')
        for param in self.efficient_net.parameters():
            param.requires_grad = True

        channels = 1280  # EfficientNet-b0 output channels
        num_patches = (7 // patch_size) ** 2
        patch_dim = channels * patch_size ** 2
        self.patch_size = patch_size

        self.pos_embedding = nn.Parameter(torch.randn(1, num_patches + 1, dim))
        self.patch_to_embedding = nn.Linear(patch_dim, dim)
        self.cls_token = nn.Parameter(torch.randn(1, 1, dim))
        self.dropout_layer = nn.Dropout(emb_dropout)

        self.transformer = Transformer(dim, depth, heads, dim_head, mlp_dim, dropout)
        self.mlp_head = nn.Sequential(
            nn.Linear(dim, mlp_dim),
            nn.ReLU(),
            nn.Linear(mlp_dim, num_classes)
        )

    def forward(self, img):
        p = self.patch_size
        x = self.efficient_net.extract_features(img)  # (B, 1280, 7, 7)
        x = rearrange(x, 'b c (h p1) (w p2) -> b (h w) (p1 p2 c)', p1=p, p2=p)
        x = self.patch_to_embedding(x)

        cls_tokens = self.cls_token.expand(img.shape[0], -1, -1)
        x = torch.cat((cls_tokens, x), dim=1)
        x += self.pos_embedding[:, :x.shape[1], :]
        x = self.dropout_layer(x)
        x = self.transformer(x)
        x = x[:, 0]
        return self.mlp_head(x)

Overwriting /kaggle/working/Combining-EfficientNet-and-Vision-Transformers-for-Video-Deepfake-Detection/efficient-vit/efficient_vit.py


In [28]:
import os
import sys
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from sklearn.metrics import accuracy_score, f1_score
from tqdm import tqdm

# Add path to efficient_vit
sys.path.append('/kaggle/working/Combining-EfficientNet-and-Vision-Transformers-for-Video-Deepfake-Detection/efficient-vit')

# Import EfficientViT model
from efficient_vit import EfficientViT

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
batch_size = 32
learning_rate = 1e-4
num_epochs = 30
num_classes = 2  # real, fake

# Image size (make sure this matches what efficient_vit expects)
image_size = 224

# Transforms (NO albumentations)
train_transforms = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

# Load dataset using ImageFolder (folder structure must be: class_name/image.jpg)
train_dataset = datasets.ImageFolder(root='/kaggle/working/cropped_faces/train', transform=train_transforms)
val_dataset = datasets.ImageFolder(root='/kaggle/working/cropped_faces/val', transform=train_transforms)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

# Create config dictionary as expected by EfficientViT
config = {
    'model': {
        'image-size': image_size,
        'patch-size': 1,   # In your model it's 1 because EfficientNet output is 7x7
        'num-classes': num_classes,
        'dim': 512,
        'depth': 6,
        'heads': 8,
        'mlp-dim': 1024,
        'emb-dim': 512,
        'dim-head': 64,
        'dropout': 0.1,
        'emb-dropout': 0.1
    }
}

# Initialize model
model = EfficientViT(config).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Directory for saving checkpoints
checkpoint_dir = "/kaggle/working/checkpoints"
os.makedirs(checkpoint_dir, exist_ok=True)

# Training loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")

    # Save checkpoint
    checkpoint_path = os.path.join(checkpoint_dir, f"model_epoch_{epoch+1}.pth")
    torch.save(model.state_dict(), checkpoint_path)
    print(f"Saved checkpoint: {checkpoint_path}")

    # Validation
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    acc = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)
    print(f"Validation Accuracy: {acc:.4f}, F1 Score: {f1:.4f}")


Loaded pretrained weights for efficientnet-b0


Epoch 1/30: 100%|██████████| 70/70 [00:19<00:00,  3.60it/s]


Epoch [1/30], Loss: 0.3066
Saved checkpoint: /kaggle/working/checkpoints/model_epoch_1.pth
Validation Accuracy: 0.5136, F1 Score: 0.6740


Epoch 2/30: 100%|██████████| 70/70 [00:19<00:00,  3.55it/s]


Epoch [2/30], Loss: 0.0732
Saved checkpoint: /kaggle/working/checkpoints/model_epoch_2.pth
Validation Accuracy: 0.7223, F1 Score: 0.7836


...

Epoch [28/30], Loss: 0.0051
Saved checkpoint: /kaggle/working/checkpoints/model_epoch_28.pth
Validation Accuracy: 0.9074, F1 Score: 0.9154


Epoch 29/30: 100%|██████████| 70/70 [00:19<00:00,  3.62it/s]


Epoch [29/30], Loss: 0.0016
Saved checkpoint: /kaggle/working/checkpoints/model_epoch_29.pth
Validation Accuracy: 0.9074, F1 Score: 0.9154


Epoch 30/30: 100%|██████████| 70/70 [00:19<00:00,  3.62it/s]


Epoch [30/30], Loss: 0.0070
Saved checkpoint: /kaggle/working/checkpoints/model_epoch_30.pth
Validation Accuracy: 0.9093, F1 Score: 0.9172
